In [214]:
using Pkg
Pkg.activate("CT")
using CT

  Activating project at `~/CT_MPS_mini/CT`


In [215]:
Pkg.status()

Project CT v0.1.0
Status `~/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
⌃ [6e4b80f9] BenchmarkTools v1.5.0
⌅ [9136182c] ITensors v0.6.16
  [682c06a0] JSON v0.21.4
⌃ [33e6dc65] MKL v0.7.0
⌃ [91a5bcdd] Plots v1.40.5
⌃ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.10
⌃ [a759f4b9] TimerOutputs v0.5.24
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [216]:
# benchmark adder_MPO
using ITensors
include("CT/src/test_adder_MPO.jl")
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO
using Random
L = 4
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# println(initial_state)
initial_state = productMPS(qubit_site, [1,2,2,1])
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)

[1, 2, 3, 4]
[1, 4, 2, 3]
[1, 3, 4, 2]


([0, 1, 0, 1], 5)

In [217]:
ram_pos = 1
qubit_site = siteinds(initial_state)
shift_bit = shift_1_3_bits[ram_phy[ram_pos]]
phy_pos = phy_ram[ram_pos]
T_left_ind = Index(2, "Carry,c=$(ram_pos-1)")
T_right_ind = Index(2, "Carry,c=$(ram_pos)")
T = create_addition_tensor_with_carry(shift_bit, qubit_site[ram_pos], prime(qubit_site[ram_pos]), T_left_ind, T_right_ind);

In [218]:
println(shift_bit)
println(T.tensor.inds)
T_array = array(T)
println(T_array[:,:,1,2])


0
((dim=2|id=925|"Qubit,Site,n=1"), (dim=2|id=925|"Qubit,Site,n=1")', (dim=2|id=847|"Carry,c=0"), (dim=2|id=426|"Carry,c=1"))
[0.0 0.0; 0.0 0.0]


In [219]:
c_in = Index(2, "Carry,c_in")
c_out = Index(2, "Carry,c_out")
T_last = create_addition_tensor_with_carry(1, qubit_site[L], prime(qubit_site[L]), c_in, c_out)

lsb_tensor = ITensor(c_in)
lsb_tensor[c_in=>1] = 1.0
T_last = T_last * lsb_tensor
println(T_last)

ITensor ord=3
Dim 1: (dim=2|id=796|"Qubit,Site,n=4")
Dim 2: (dim=2|id=796|"Qubit,Site,n=4")'
Dim 3: (dim=2|id=911|"Carry,c_out")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×2
[:, :, 1] =
 0.0  1.0
 0.0  0.0

[:, :, 2] =
 0.0  0.0
 1.0  0.0


In [220]:
using ITensors

# Create indices with explicit dimensions
i = Index(3, "i")
j = Index(4, "j") 
k = Index(2, "k")
T = randomITensor(i, j, k)

# Check the tensor structure
println("Tensor indices: ", inds(T))
println("Index dimensions: ", [dim(idx) for idx in inds(T)])

# Make sure the value is within bounds (1-indexed in Julia)
# For index i with dimension 3, valid values are 1, 2, 3
T_fixed = T[i => 1]  # or 2, or 3

Tensor indices: ((dim=3|id=807|"i"), (dim=4|id=423|"j"), (dim=2|id=658|"k"))
Index dimensions: [3, 4, 2]


LoadError: BoundsError: attempt to access Tuple{Int64} at index [0]

In [ ]:
include("global_adder_passthrough.jl")
initial_state = global_adder_folded(initial_state, ram_phy, shift_1_3_bits)


Index{Int64}[(dim=2|id=309|"Carry,c=0"), (dim=2|id=71|"Qubit,Site,n=1"), (dim=1|id=329|"Link,l=1")]
Index{Int64}[(dim=2|id=968|"Carry,c=1"), (dim=1|id=852|"Link,l=1"), (dim=2|id=82|"Qubit,Site,n=2"), (dim=2|id=804|"Carry,c=2"), (dim=1|id=66|"Link,l=2")]
ITensor ord=2
Dim 1: (dim=2|id=71|"Qubit,Site,n=1")
Dim 2: (dim=1|id=852|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×1
 0.6484197773255047
 0.6484197773255047
Cut between sites 1 and 2: [0.9999999999999997]
Cut between sites 2 and 3: [0.9999999999999996, 0.0]
Cut between sites 3 and 4: [0.9999999999999996]
Cut between sites 1 and 2: [0.0]
Cut between sites 2 and 3: [0.0, 0.0]
Cut between sites 3 and 4: [0.0, 0.0]


MPS
[1] ((dim=2|id=71|"Qubit,Site,n=1"), (dim=1|id=852|"Link,l=1"))
[2] ((dim=4|id=459|"Link,l=2"), (dim=2|id=82|"Qubit,Site,n=2"), (dim=1|id=852|"Link,l=1"))
[3] ((dim=4|id=784|"Link,l=3"), (dim=4|id=459|"Link,l=2"), (dim=2|id=712|"Qubit,Site,n=3"))
[4] ((dim=4|id=784|"Link,l=3"), (dim=2|id=959|"Qubit,Site,n=4"))


In [ ]:
# function print_mpo_as_matrix(mpo, L)
#     println("\nMPO as matrix:")
    
#     # Contract the entire MPO to get the matrix representation
#     mpo_tensor = contract(mpo)
    
#     # The tensor has indices (s1', s2', ..., sL', s1, s2, ..., sL)
#     # We need to reshape it into a matrix where the first L indices are rows
#     # and the last L indices are columns
    
#     # Get the tensor as an array
#     tensor_array = Array(mpo_tensor, inds(mpo_tensor)...)
    
#     # Reshape to matrix: first 2^L entries are rows, next 2^L are columns
#     matrix = reshape(tensor_array, 2^L, 2^L)
    
#     # Create basis labels
#     basis_labels = []
#     for i in 0:2^L-1
#         bits = string(i, base=2, pad=L)
#         push!(basis_labels, "|$bits⟩")
#     end
    
#     # Print header
#     print("      ")
#     for j in 1:2^L
#         print("$(basis_labels[j])  ")
#     end
#     println()
    
#     # Print matrix rows
#     for i in 1:2^L
#         print("$(basis_labels[i]) ")
#         for j in 1:2^L
#             val = abs(matrix[i, j])
#             if val > 0.5
#                 print("  1   ")
#             else
#                 print("  0   ")
#             end
#         end
#         println()
#     end
    
#     return matrix
# end

# print_mpo_as_matrix(mpo, 4)

In [ ]:
# function print_mpo_as_matrix(mpo, L)
#     println("\nMPO as matrix:")
    
#     # Contract the entire MPO to get the matrix representation
#     mpo_tensor = contract(mpo)
    
#     # The tensor has indices (s1', s2', ..., sL', s1, s2, ..., sL)
#     # We need to reshape it into a matrix where the first L indices are rows
#     # and the last L indices are columns
    
#     # Get the tensor as an array
#     tensor_array = Array(mpo_tensor, inds(mpo_tensor)...)
    
#     # Reshape to matrix: first 2^L entries are rows, next 2^L are columns
#     matrix = reshape(tensor_array, 2^L, 2^L)
    
#     # Create basis labels
#     basis_labels = []
#     for i in 0:2^L-1
#         bits = string(i, base=2, pad=L)
#         push!(basis_labels, "|$bits⟩")
#     end
    
#     # Print header
#     print("      ")
#     for j in 1:2^L
#         print("$(basis_labels[j])  ")
#     end
#     println()
    
#     # Print matrix rows
#     for i in 1:2^L
#         print("$(basis_labels[i]) ")
#         for j in 1:2^L
#             val = abs(matrix[i, j])
#             if val > 0.5
#                 print("  1   ")
#             else
#                 print("  0   ")
#             end
#         end
#         println()
#     end
    
#     return matrix
# end

# print_mpo_as_matrix(mpo, 4)

In [ ]:
# function print_mpo_as_matrix(mpo, L)
#     println("\nMPO as matrix:")
    
#     # Contract the entire MPO to get the matrix representation
#     mpo_tensor = contract(mpo)
    
#     # The tensor has indices (s1', s2', ..., sL', s1, s2, ..., sL)
#     # We need to reshape it into a matrix where the first L indices are rows
#     # and the last L indices are columns
    
#     # Get the tensor as an array
#     tensor_array = Array(mpo_tensor, inds(mpo_tensor)...)
    
#     # Reshape to matrix: first 2^L entries are rows, next 2^L are columns
#     matrix = reshape(tensor_array, 2^L, 2^L)
    
#     # Create basis labels
#     basis_labels = []
#     for i in 0:2^L-1
#         bits = string(i, base=2, pad=L)
#         push!(basis_labels, "|$bits⟩")
#     end
    
#     # Print header
#     print("      ")
#     for j in 1:2^L
#         print("$(basis_labels[j])  ")
#     end
#     println()
    
#     # Print matrix rows
#     for i in 1:2^L
#         print("$(basis_labels[i]) ")
#         for j in 1:2^L
#             val = abs(matrix[i, j])
#             if val > 0.5
#                 print("  1   ")
#             else
#                 print("  0   ")
#             end
#         end
#         println()
#     end
    
#     return matrix
# end

# print_mpo_as_matrix(mpo, 4)

In [ ]:
# function print_mpo_as_matrix(mpo, L)
#     println("\nMPO as matrix:")
    
#     # Contract the entire MPO to get the matrix representation
#     mpo_tensor = contract(mpo)
    
#     # The tensor has indices (s1', s2', ..., sL', s1, s2, ..., sL)
#     # We need to reshape it into a matrix where the first L indices are rows
#     # and the last L indices are columns
    
#     # Get the tensor as an array
#     tensor_array = Array(mpo_tensor, inds(mpo_tensor)...)
    
#     # Reshape to matrix: first 2^L entries are rows, next 2^L are columns
#     matrix = reshape(tensor_array, 2^L, 2^L)
    
#     # Create basis labels
#     basis_labels = []
#     for i in 0:2^L-1
#         bits = string(i, base=2, pad=L)
#         push!(basis_labels, "|$bits⟩")
#     end
    
#     # Print header
#     print("      ")
#     for j in 1:2^L
#         print("$(basis_labels[j])  ")
#     end
#     println()
    
#     # Print matrix rows
#     for i in 1:2^L
#         print("$(basis_labels[i]) ")
#         for j in 1:2^L
#             val = abs(matrix[i, j])
#             if val > 0.5
#                 print("  1   ")
#             else
#                 print("  0   ")
#             end
#         end
#         println()
#     end
    
#     return matrix
# end

# print_mpo_as_matrix(mpo, 4)

In [ ]:
# # Create 2D identity matrices to compare against
# expected_identity = [1.0 0.0; 0.0 1.0]

# # Extract T[1,:,:,1] as a matrix
# slice_1 = zeros(2, 2)
# for i in 1:2, j in 1:2
#     slice_1[i, j] = T[i1 => 1, i2 => i, i3 => j, i4 => 1]
# end

# # Extract T[2,:,:,2] as a matrix  
# slice_2 = zeros(2, 2)
# for i in 1:2, j in 1:2
#     slice_2[i, j] = T[i1 => 2, i2 => i, i3 => j, i4 => 2]
# end

# println("T[1,:,:,1] =")
# display(slice_1)
# println("Expected identity =")
# display(expected_identity)
# println("Are they equal? ", slice_1 ≈ expected_identity)

# println("\nT[2,:,:,2] =")
# display(slice_2)
# println("Are they equal? ", slice_2 ≈ expected_identity)

In [ ]:
# function print_mpo_as_matrix(mpo, L)
#     println("\nMPO as matrix:")
    
#     # Contract the entire MPO to get the matrix representation
#     mpo_tensor = contract(mpo)
    
#     # The tensor has indices (s1', s2', ..., sL', s1, s2, ..., sL)
#     # We need to reshape it into a matrix where the first L indices are rows
#     # and the last L indices are columns
    
#     # Get the tensor as an array
#     tensor_array = Array(mpo_tensor, inds(mpo_tensor)...)
    
#     # Reshape to matrix: first 2^L entries are rows, next 2^L are columns
#     matrix = reshape(tensor_array, 2^L, 2^L)
    
#     # Create basis labels
#     basis_labels = []
#     for i in 0:2^L-1
#         bits = string(i, base=2, pad=L)
#         push!(basis_labels, "|$bits⟩")
#     end
    
#     # Print header
#     print("      ")
#     for j in 1:2^L
#         print("$(basis_labels[j])  ")
#     end
#     println()
    
#     # Print matrix rows
#     for i in 1:2^L
#         print("$(basis_labels[i]) ")
#         for j in 1:2^L
#             val = abs(matrix[i, j])
#             if val > 0.5
#                 print("  1   ")
#             else
#                 print("  0   ")
#             end
#         end
#         println()
#     end
    
#     return matrix
# end

# print_mpo_as_matrix(mpo, 4)

In [ ]:
# # debug 
# # haining
# using .CT: add1, power_mpo
# add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site)
# add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)])

# # tao
# # mpo, bond_dims, dense_mat, A_tensor = mpo_adder_1_3(L, qubit_site, ram_phy, phy_ram, phy_list, folded);
# shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# mpo = create_binary_addition_mpo(shift_1_3_bits, qubit_site, phy_ram, phy_list, ram_phy)

# result_h = initial_state * add1_3
# result_t = initial_state * mpo
# n = 3
# println(result_h[n])
# println(result_t[n])